<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dataset_class/Blue-Block/blue_semantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 3.3 MB/s 
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53924 sha256=0471cdbacc96ff5fe10473c8d4f6b5aaf5eb25926885ccdc497bff7bb38250cf
  Stored in directory: /root/.cache/pip/wheels/78/ae/2e/5ce789557ca59281a463dabe9d6522e429be240322148e5379
Successfully built docx


In [4]:
!pip install exceptions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement exceptions (from versions: none)
ERROR: No matching distribution found for exceptions


In [6]:
import pandas as pd
import os
#from docx import Document
#from docx.oxml.ns import qn
#from docx.shared import Pt
import numpy as np
import re
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
save_file_csv = 'blue2.csv'
file_csv =  'blue.csv'
file_dir = '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block' #Newly stored class address


In [9]:
fileList = []
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
#       print(root) #current directory path
#       print(files) #All non-directory subfiles in the current path
         for file in files:
            if file[-4:] == '.txt':
                fileList.append(os.path.join(root, file))

In [10]:
file_name(file_dir)

In [11]:
fileList

['/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Ground.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/PowerUp.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Locator.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/InfoWindow.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/ResourceManager.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/GroundType.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/InfoPanel.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Menu.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Settings.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Main.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Options.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Game.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Human.txt']

In [12]:
# stopwords
stopwords = ['abstract', 'java','assert', 'boolean', 'break', 'byte', 'case', 'catch', 'class','char', 'continue', 'default', 'do', 'double', 'else', 'enum', 'extends', 'final', 'finally', 'float', 'for', 'if', 'implements', 'import', 'int', 'interface', 'long', 'native', 'new', 'package','private','public', 'protected',  'return', 'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this', 'throw', 'transient', 'try', 'void', 'volatile', 'while']
dicts = {'\n': '','!': '', '"': '','：':'', '#': '', '$': '', '%': '', '&': '', "'": '', '(': '', ')': '', '*': '', '+': '', ',': '', '-': '', '.': '', '/': '', ':': '', ';': '', '<': '', '=': '', '>': '', '?': '', '@': '', '[': '', '\\': '', ']': '', '^': '', '_': '', '`': '', '{': '', '|': '', '}': '', '~': ''}
# stopwords = {'abstract':'', 'java':'','assert':'', 'boolean':'', 'break':'', 'byte':'', 'case':'', 'catch':'', 'class':'','char':'', 'continue':'', 'default':'', 'do':'', 'double':'', 'else':'', 'enum':'', 'extends':'', 'final':'', 'finally':'', 'float':'', 'for':'', 'if':'', 'implements':'', 'import':'', 'int':'', 'interface':'', 'long':'', 'native':'', 'new':'', 'package':'','private':'','public':'', 'protected':'',  'return':'', 'short':'', 'static':'', 'strictfp':'', 'super':'', 'switch':'', 'synchronized':'', 'this':'', 'throw':'', 'transient':'', 'try':'','void':'', 'volatile':'', 'while':''}

In [13]:
def lm_find_unchinese(line):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",line) #Exclude Chinese characters
    unchinese = re.sub('[{}]'.format(punctuation),"",unchinese) #Exclude Chinese characters
    #print("unchinese:",unchinese)
    return unchinese

In [14]:
def del_stopwords(line):
#     for line in file.readlines():
#     dicts = {i: '' for i in punctuation}
    punc_table = str.maketrans(dicts)
    new_line = line.translate(punc_table)

#     for word in stopwords:
#           new_line = new_line.replace(word, '')

    return new_line


In [15]:
index = []
index_all = []
class_name = []
Brain_Class = []
for i, f in enumerate(fileList):
    with open(f,encoding='gb18030',errors='ignore') as file:
        list = []
        list2 = []
        is_brain = 0
        filename = os.path.basename(file.name)
        name = filename[:-4]
        class_name.append(name)
        list.append(name)
        list2.append(name)
        
        for line in file.readlines():
            line = lm_find_unchinese(line)
            line = del_stopwords(line)
            values = line.split(' ')
            coefs = values[1:-1]
            coefs = [i for i in coefs if i != '']
            keyword_brain = 'hongshuai'            

            if  keyword_brain in line:
                is_brain = 1
            if len(coefs) != 0:
                if coefs != ' ':
                    if coefs != '\n':
                        for i in coefs:
                            list.append(i)

        index.append(list)
        
        if is_brain == 0:
            list2.append(False)
        else :
            list2.append(True)
            print('T')
        
#         print (list2)
        
        Brain_Class.append(list2)

T


In [16]:
Brain_Class = np.array(Brain_Class)
Brain_Class

array([['Ground', 'False'],
       ['PowerUp', 'True'],
       ['Locator', 'False'],
       ['InfoWindow', 'False'],
       ['ResourceManager', 'False'],
       ['GroundType', 'False'],
       ['InfoPanel', 'False'],
       ['Menu', 'False'],
       ['Settings', 'False'],
       ['Main', 'False'],
       ['Options', 'False'],
       ['Game', 'False'],
       ['Human', 'False']], dtype='<U15')

In [17]:
def additem(list):
    if len(list)<3:
        q = len(list)-1
        str = '_1'
        str2 = '_2'
        item = ''.join([list[q], str])
        item_2 = ''.join([list[q], str2])
        # item = list[q]
        list.append(item)
        list.append(item_2)
        # print(list)
    return list

for item in index_all:
    # print(item)
    item = additem(item)
    # print(item)

In [18]:
news_df = np.array(index,dtype=object)
print(news_df)

[list(['Ground', 'GroundType', 'return', 'int', 'return', 'int', 'return', 'GroundGame', 'gameint', 'xint', 'yGroundType', 'void', 'void', 'SetGroundTypeGroundType', 'if', 'Type', 'null'])
 list(['PowerUp', 'boolean', 'return', 'void', 'int', 'int', 'void', 'NewPositionint', 'xint', 'Ground', 'GroundType', 'if', 'grtIsWall', 'grtIsDeadly', 'gameLocatorGetHumanxy', 'null', 'int', 'int', 'if', 'grtIsPoison', 'PowerUpGame', 'void', 'TakeEffectHuman', 'SystemoutprintlnPowerUp', 'GetType', 'Human', 'human2', 'gameHumans'])
 list(['Locator', 'void', 'backgroundColorint', 'pos1int', 'Ground', 'if', 'groundGetType', 'GroundTypeFloor', 'void', 'checkKillsHuman', 'if', 'SettingsEnablePlayerKills', 'for', 'Human', 'other', 'gameHumans', 'if', 'other', 'human', 'otherGetX', 'humanGetX', 'otherGetY', 'humanGetY', 'void', 'checkPowerupsHuman', 'PowerUp', 'if', 'PU', 'null', 'Ground', 'GetGroundint', 'xint', 'if', 'x', '0', 'y', '0', 'x', 'gameWidth', 'y', 'gameHeight', 'return', 'return', 'Human', '

In [19]:
# #training phase
def f_lsa(news_df, n):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(news_df)
    # print(vectorizer.get_feature_names())#Get the vocabulary found by the model directly analyzing the data（collection of words above）
    # print(X.toarray())# Printing X directly outputs the position of each word
    #     print(X)

    n_cap = 1
    svd_model = TruncatedSVD(n_components=n_cap)  # Latent semantic analysis, set 4 topics
    X2 = svd_model.fit_transform(X)  # train and convert

    # print("--------lsa Singular Value---------")
    # print(svd_model.singular_values_)
    # print("--------Representation of text in topic vector space Left singular matrix---------")
    # print(X2)

    terms = vectorizer.get_feature_names()
    # print("-------- Topics and keywords ---------")
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
        # print("Topic " + str(i) + ": ")
        for t in sorted_terms:
            print(t[0])
    print(" ")

    D = svd_model.components_
    T = X2
    S = np.diagflat(svd_model.singular_values_)

    X = np.dot(T, S)
    X = np.dot(X, D)
    c = np.dot(X, X.T)

    acsm = None
    sum = 0
    for i in c:
        for j in i:
            if j < 0:
                j = 0
            sum += j
        acsm = sum / n / n
    # print("-------------acsm----------------------")
    # print(acsm)

    return acsm

In [20]:
q = 0
acsm= []
for i in index:
    lists = []
    print("*************",i,"****************")
    # print("------------------------------------------------------------------")
    for j in i:
        lists.append(j)
    # lists = deleteDuplicatedElementFromList(lists)
    if len(lists) > 1:
        acsm.append(f_lsa(lists,len(i)))
    else :
        acsm.append(1)
#

# print(type(class_name))
# print(type(acsm))

************* ['Ground', 'GroundType', 'return', 'int', 'return', 'int', 'return', 'GroundGame', 'gameint', 'xint', 'yGroundType', 'void', 'void', 'SetGroundTypeGroundType', 'if', 'Type', 'null'] ****************
return
ygroundtype
xint
gameint
groundtype
type
if
 
************* ['PowerUp', 'boolean', 'return', 'void', 'int', 'int', 'void', 'NewPositionint', 'xint', 'Ground', 'GroundType', 'if', 'grtIsWall', 'grtIsDeadly', 'gameLocatorGetHumanxy', 'null', 'int', 'int', 'if', 'grtIsPoison', 'PowerUpGame', 'void', 'TakeEffectHuman', 'SystemoutprintlnPowerUp', 'GetType', 'Human', 'human2', 'gameHumans'] ****************
int
gamelocatorgethumanxy
groundtype
gamehumans
systemoutprintlnpowerup
null
powerupgame
 
************* ['Locator', 'void', 'backgroundColorint', 'pos1int', 'Ground', 'if', 'groundGetType', 'GroundTypeFloor', 'void', 'checkKillsHuman', 'if', 'SettingsEnablePlayerKills', 'for', 'Human', 'other', 'gameHumans', 'if', 'other', 'human', 'otherGetX', 'humanGetX', 'otherGetY', '

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


************* ['Options', 'Optionsfinal', 'Menu', 'optionsnew', 'JFrameBlue', 'Block', 'optionssetBoundsResourceManagerScreenWidth', '2', '225ResourceManagerScreenHeight', '2', 'Font', 'fontnew', 'JLabel', 'widthLabelnew', 'JLabelResourceManagerLanguageStringsgetWidth', 'textFieldWidthnew', 'JLabel', 'heightLabelnew', 'JLabelResourceManagerLanguageStringsgetHeight', 'textFieldHeightnew', 'JLabel', 'playerTextnew', 'JLabelResourceManagerLanguageStringsgetPlayers', 'playerCountSelectornew', 'for', 'int', 'i1', 'i', '5', 'i', 'playerCountSelectoraddi', 'playerCountSelectorselectplayerCountSelector', 'JLabel', 'powerUpLabelnew', 'JLabelResourceManagerLanguageStringsgetPowerups', 'powerUpCountSelectornew', 'for', 'int', 'i0', 'i', '11', 'i', 'powerUpCountSelectoraddi', 'powerUpCountSelectorselectSettingsPowerupCount', 'mouseCheckboxnew', 'killCheckboxnew', 'Font', 'harringtonnew', 'savenew', 'saveaddActionListenernew', 'Override', 'public', 'void', 'actionPerformed', 'ActionEvent', 'resetne

In [21]:
c = []
for i in range(len(class_name)):
      for j in range(len(acsm)):
        if i==j:
            t=[class_name[i],acsm[j]]
            c.append(t)

# c = list(zip(class_name,acsm))
# print(type(c))
c = np.array(c)
# print(c)
# print(c[0][0])

In [23]:
c

array([['Ground', '0.09350965437944614'],
       ['PowerUp', '0.08166457757109888'],
       ['Locator', '0.024461898435713104'],
       ['InfoWindow', '0.008331384216452646'],
       ['ResourceManager', '0.029594318048441197'],
       ['GroundType', '0.6666666666666667'],
       ['InfoPanel', '0.06173598534173795'],
       ['Menu', '0.8145419273893149'],
       ['Settings', '0.02463108093757165'],
       ['Main', '0.16748079210765243'],
       ['Options', '0.01954373444961729'],
       ['Game', '0.07998493375146248'],
       ['Human', '0.3429356522942449']], dtype='<U32')

In [24]:
df =pd.read_csv("/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/blue.csv") 

In [25]:
x= df.iloc[:, 0]

In [26]:
Brain_Class

array([['Ground', 'False'],
       ['PowerUp', 'True'],
       ['Locator', 'False'],
       ['InfoWindow', 'False'],
       ['ResourceManager', 'False'],
       ['GroundType', 'False'],
       ['InfoPanel', 'False'],
       ['Menu', 'False'],
       ['Settings', 'False'],
       ['Main', 'False'],
       ['Options', 'False'],
       ['Game', 'False'],
       ['Human', 'False']], dtype='<U15')

In [27]:
for i in range(0,len(Brain_Class)):
    if 'False' == Brain_Class[i][1]:
        q = 1
    else :
        print('*')

*


In [28]:
#add new column
ngList=[] #Prepare a list and store the data of the new column in it

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(c[i][0])
        st = st[1:]
#         print(st)
        if index == st:
            ngList.append(c[i][1])
            q = 0
#             print('*')
    if q==1 :
        ngList.append(1)

In [29]:
brainList=[] #Prepare a list and store the data of the new column in it

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(Brain_Class[i][0])
        st = st[1:]
        if index == st:
            brainList.append(Brain_Class[i][1])
            q = 0
    if q==1 :
        brainList.append('0')
        
        
print(len(brainList))
print(len(x))

14
14


In [30]:
column=df.columns.tolist()

In [31]:
df['acsm'] = ngList 
df['is_brain'] = brainList 
df.to_csv(save_file_csv,mode = 'a',index =False)

In [32]:
df.head()

,NAME,AMW,ATDF,BOVM,BUR,CBO,CC,CM,CRIX,DAC,...,NOA,NOAM,NOD,NOM,NOPA,TCC,WMC,WOC,acsm,is_brain
0,Locator,4.78,18,0,1,7,3,4,2.5,1,...,1,0,0,9,0,0.57,43,1.00,1,0
1,InfoPanel,3.33,5,0,1,3,1,1,0.0,0,...,5,0,0,3,0,1.00,10,1.00,1,0
2,1,1.00,0,0,1,2,0,0,0.0,0,...,0,0,0,1,0,1.00,1,1.00,1,0
3,Main,1.94,6,0,1,5,6,9,0.0,2,...,7,0,0,17,3,0.11,33,0.83,1,0
4,InfoWindow,1.67,8,0,1,5,1,5,0.0,4,...,6,0,0,3,1,0.00,5,0.67,1,0
